In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [ ]:
from langgraph.checkpoint.memory import MemorySaver
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.messages import BaseMessage, SystemMessage
from langgraph.graph import START, MessagesState, StateGraph
import time

llm = ChatOpenAI(temperature=0)

workflow = StateGraph(state_schema=MessagesState)

def call_model(state: MessagesState) -> MessagesState:
    response = llm.invoke(state["messages"])
    return {"messages": response}

workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

memory = MemorySaver()

app = workflow.compile(checkpointer=memory)

change = 50

In [ ]:
from langgraph.checkpoint.memory import MemorySaver
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.messages import BaseMessage, SystemMessage
from langgraph.graph import START, MessagesState, StateGraph
import time

llm = ChatOpenAI(temperature=0)

workflow = StateGraph(state_schema=MessagesState)

def call_model(state: MessagesState) -> MessagesState:
    response = llm.invoke(state["messages"])
    return {"messages": response}

workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

memory = MemorySaver()

app = workflow.compile(checkpointer=memory)

change = 50

In [216]:
system_template = """
You are a python code quality evaluator. You will follow the given guidelines sequentially.
1. First, you will receive a question in next prompt, so you will wait for the question.
2. Then you will receive a code snippet by user periodically. 
3. You have to evaluate the quality of code provided by user using the following steps:
    - There are following return types:
        a. Incomplete or erroneous
            Meaning: Code is incomplete or it is producing errors (it cannot be run succesfully).
        b. Incorrect but better
            Meaning: Code is incorrect but better than previous code snippet in terms of syntax, logic and correctness. This means the user is moving in right direction towards the correct code.
        c. Incorrect and worse
            Meaning: Code is incorrect and worse than previous code snippet in terms of syntax, logic and correctness. This means the user is moving in wrong direction away from the correct code.
        d. Incorrect and same as previous code snippet
            Meaning: Code is incorrect and same as previous code snippet and there is no improvement in the code.
        e. Partially correct
            Meaning: Code is partially correct. It covers some of the requirements from the question but not all.
        f. Correct but can be improved
            Meaning: Syntax is correct, code does not produce any error on running it and output is as expected BUT the code can be improved (improvements can be in terms of code optimization, better logic, better variable names, better comments, etc.)
        g. Correct
            Meaning: Syntax is correct, code does not produce any error on running it and output is as expected. Code is following the guidelines provided in the question. There is no redundancy in the code. Code is optimized and well written.
4. Give one of these return types as output. And write one line evaluation comment on separate line.
5. Add an approximate score out of 100 on separate line as the quality of the code snippet provided by user. This score should be based on the quality of code snippet and how well it is following the guidelines provided in the question.
6. Your main goal is to identify if user is improving their code or not.
7. Restraint from giving the solution or suggestion or hints.
"""

system_question = """
Questions: {question}
"""

user_template = """
{code}
"""

change += 1

In [217]:
user_prompt = ChatPromptTemplate(["user",user_template])
system_question_prompt = ChatPromptTemplate(["system",system_question])

In [218]:
ACTIVE_THREAD_ID = f"test{change}"

config = {"configurable":{"thread_id":ACTIVE_THREAD_ID}}

def get_code(filename:str) -> str:
    with open(filename) as f:
        s = f.read()
        return s
    
def get_user_code_snippet() -> list[BaseMessage]:
    code_snippet = get_code("test.py")
    return user_prompt.invoke({"code": code_snippet}).to_messages()


def get_output(messages: list[BaseMessage]):
    output = app.invoke({"messages": messages}, config)
    output["messages"][-1].pretty_print()

In [219]:
system_message = SystemMessage(system_template)
get_output(system_message)

question = "Write a program that takes input first name and last name from the user and passes it to the function greeting_full_name to print the full name of the user with a greeting."
system_question_message = system_question_prompt.invoke({"question":question}).to_messages()
get_output(system_question_message)

================================== Ai Message ==================================

Waiting for the question.
================================== Ai Message ==================================

Incomplete or erroneous.
Function `greeting_full_name` is missing and the code does not take input from the user.


In [220]:
while True:
    user_message = get_user_code_snippet()
    get_output(user_message)
    time.sleep(5)

================================== Ai Message ==================================

Incomplete or erroneous.
The code provided is empty and does not contain any implementation.
================================== Ai Message ==================================

Incorrect but better
Added input statements to take first name and last name from the user, but still missing the `greeting_full_name` function implementation. 

Approximate score: 30/100
================================== Ai Message ==================================

Incorrect and same as previous code snippet

Approximate score: 30/100
================================== Ai Message ==================================

Partially correct
Added the `greeting_full_name` function but missing the implementation inside the function to print the full name with a greeting.

Approximate score: 50/100
================================== Ai Message ==================================

Partially correct
Implemented the `greeting_full_name` functio

KeyboardInterrupt: 

6